In [1]:
import pandas as pd
import csv
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import rasterio
import rasterio.mask
from rasterio.plot import show_hist
from rasterio.plot import show

import rasterstats
from descartes import PolygonPatch

from matplotlib import pyplot as plt
import geojson
from shapely.geometry import Point, mapping, shape
from fiona import collection
import fiona
import geopandas as gpd
import geoplot 

from shape_generator import Shape_Generator, Population_Calculator

In [2]:
rasterio.gdal_version()

'3.0.4'

## Reading in the cyclone dataset
Important: This notebook only works if all the files are setup according to the GitHub repository!

In [8]:
# reading in the cyclone data
data_folder = Path.cwd().parent.parent.joinpath('datasets')
print(data_folder)
input_filename = 'epac-atl.csv'
input_file = Path(data_folder).joinpath(input_filename)
print(input_file)

G:\Omdena\wfpdisrel\#task1-datacollection\datasets\epac-atl.csv


In [18]:
df_input = pd.read_csv(input_file)
df_input.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id,name,time,year,lat,long,...,NE_50,SE_50,SW_50,NW_50,NE_64,SE_64,SW_64,NW_64,max_50,max_64
0,0,0,0,0,AL0188,ALBERTO,80518,1988,32.0,-77.5,...,0,0,0,0,0,0,0,0,0.0,0.0
1,1,1,1,1,AL0188,ALBERTO,80600,1988,32.8,-76.2,...,0,0,0,0,0,0,0,0,0.0,0.0
2,2,2,2,2,AL0188,ALBERTO,80606,1988,34.0,-75.2,...,0,0,0,0,0,0,0,0,0.0,0.0
3,3,3,3,3,AL0188,ALBERTO,80612,1988,35.2,-74.6,...,0,0,0,0,0,0,0,0,0.0,0.0
4,4,4,4,4,AL0188,ALBERTO,80618,1988,37.0,-73.5,...,0,0,0,0,0,0,0,0,0.0,0.0


## Create list of cyclones by name and year

In [19]:
cyclones = df_input.groupby(['id','name','year'], as_index = False).sum()

In [20]:
cyclones.tail()

,id,name,year,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,time,lat,long,...,NE_50,SE_50,SW_50,NW_50,NE_64,SE_64,SW_64,NW_64,max_50,max_64
720,EP2216,OTTO,2016,740178,740178,740178,256698,4038624,391.6,-2963.7,...,310,300,250,300,310,300,250,300,310.0,310.0
721,EP2218,TARA,2018,216775,216775,216775,82475,1015302,179.7,-1040.7,...,20,20,20,20,20,20,20,20,20.0,20.0
722,EP2318,VINCENT,2018,412148,412148,412148,156978,1939862,275.7,-1827.4,...,0,0,0,0,0,0,0,0,0.0,0.0
723,EP2418,WILLA,2018,477675,477675,477675,182215,2246986,385.8,-2315.6,...,530,530,370,470,530,530,370,470,530.0,530.0
724,EP2518,XAVIER,2018,630402,630402,630402,240932,3205152,492.6,-3165.6,...,180,0,0,160,180,0,0,160,180.0,180.0


In [21]:
cyclones[['name', 'year']]

,name,year
0,ALLISON,2001
1,ARTHUR,2002
2,ANA,2003
3,ALEX,2004
4,ARLENE,2005
...,...,...
720,OTTO,2016
721,TARA,2018
722,VINCENT,2018
723,WILLA,2018


## Example on how to use the Shape_Generator to create shapefiles for single cyclones


In [22]:
cyclone_name = 'IVAN'
cyclone_year = 2004

In [23]:
obj = Shape_Generator(input_file, cyclone_name, cyclone_year, 'max_50')
my_cyclone_df = obj.generate_buffered_file(radius=50, generateFile=False) # To create the shapefile set generateFile to True

In [24]:
my_cyclone_df['geometry']

name  year
IVAN  2004    POLYGON ((-88.40278 26.87101, -88.63836 27.812...
Name: geometry, dtype: geometry

In [25]:
my_cyclone_df['geometry'].values

<GeometryArray>
Length: 1, dtype: geometry

## Generate shapefiles / population data for all cyclones

In [27]:
windspeed = 'max_50'
cyclone_data = []
for i in range (10):                  
#for i in range (len(cyclones)):             ## Change here to get output for all cyclones
    cyclone = {}
    cyclone_name = (cyclones.iloc[i]['name'])
    cyclone_year = (cyclones.iloc[i]['year'])

    i +=1
    cyclone['name'] = cyclone_name
    cyclone['year'] = cyclone_year
    my_pop = Population_Calculator(input_file, cyclone_name, cyclone_year, windspeed)
    my_cyclone_df = my_pop.generate_buffered_file(radius=50, generateFile=False)
    cyclone['pop_'+windspeed] = my_pop.get_population()  
    cyclone['max_50_shape'] = my_cyclone_df['geometry'].values[0]  ### call the correct functions here!
    cyclone_data.append(cyclone)
    print (cyclone_name, cyclone_year, cyclone['pop_'+windspeed])
    

ALLISON 2001 16938104.0
ARTHUR 2002 153616.28125
ANA 2003 None
ALEX 2004 11284.4873046875
ARLENE 2005 11822155.0
ALBERTO 2006 4862461.5
ANDREA 2007 None
ARTHUR 2008 444745.90625
ALEX 2010 1310176.375
ARLENE 2011 1518704.125


In [28]:
cyclone_data

[{'name': 'ALLISON',
  'year': 2001,
  'pop_max_50': 16938104.0,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c60280>},
 {'name': 'ARTHUR',
  'year': 2002,
  'pop_max_50': 153616.28125,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18eefdbcee0>},
 {'name': 'ANA',
  'year': 2003,
  'pop_max_50': None,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c60730>},
 {'name': 'ALEX',
  'year': 2004,
  'pop_max_50': 11284.4873046875,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c6a880>},
 {'name': 'ARLENE',
  'year': 2005,
  'pop_max_50': 11822155.0,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c44f70>},
 {'name': 'ALBERTO',
  'year': 2006,
  'pop_max_50': 4862461.5,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c6a130>},
 {'name': 'ANDREA',
  'year': 2007,
  'pop_max_50': None,
  'max_50_shape': <shapely.geometry.polygon.Polygon at 0x18e93c44550>},
 {'name': 'ARTHUR',
  'year': 2008,
  'pop_max_50': 4447

## Create dataframe from cyclone data

In [29]:
cyclone_pop_df = pd.DataFrame(cyclone_data)
cyclone_pop_df.head()

,name,year,pop_max_50,max_50_shape
0,ALLISON,2001,1.693810e+07,POLYGON ((-95.74874378344926 28.49157558595935...
1,ARTHUR,2002,1.536163e+05,POLYGON ((-75.30031405575329 35.33012587904549...
2,ANA,2003,NaN,POLYGON ((-65.35108826189396 30.80364807980347...
3,ALEX,2004,1.128449e+04,POLYGON ((-79.26007861032321 30.96274246956224...
4,ARLENE,2005,1.182216e+07,POLYGON ((-84.34915764205977 17.44107845765113...


In [30]:
cyclone_pop_df.to_pickle("../data/fake_population.pkl")